In [10]:
import pandas as pd
import numpy as np
import datetime
import os.path
import csv
from pathlib import Path
%matplotlib inline
import matplotlib.pyplot as plt

## Function: Get tickers for years

In [2]:
def get_tickers_for_years(years,sector):
    """pass me a list of years [2013,2015,...], and right, ZERO parameter or error checking"""
    tech_csv = Path(f"./{sector}SectorTickers-between-2015-2018.csv")
    #print(type(years))
    tech_tickers_df = pd.read_csv(tech_csv)
    #print(tech_tickers_df.head())
    all_tickers = []
    
    for year in years:
        #print(type(year))
        ticker_list = tech_tickers_df[year].tolist()
        ticker_list = [x for x in ticker_list if str(x) != "nan"]
        all_tickers.extend(ticker_list)
    
    return np.array(all_tickers)

## Function: Calculate the top performing tickers in range

In [11]:
def combined_df_for_timeframe(all_prices_df,ticker_list,day_start,day_end,sector):
    """Explain what I do here ;) """
    top_dogs = None
    
    print(f"\n\nRunning combined_df_for_timeframe wtih start: {day_start} and end: {day_end}\n\n")
    
    range_df = all_prices_df[(all_prices_df['Date'] >= day_start) & (all_prices_df['Date'] <= day_end)]

    # Create empty dataframe with dates
    prices_df = range_df[range_df['Ticker']=='A']
    prices_df = prices_df[['Date','Adj Close']].copy()
    prices_df = prices_df.drop(columns=["Adj Close"])
    prices_df = prices_df.set_index('Date')
    
    for ticker in ticker_list:
        if ticker in ticker_list:
            # I can't say I understand this bit... [Will]
            ticker_df = range_df[range_df['Ticker']==ticker]
            ticker_df = ticker_df[['Date', 'Adj Close']].copy()
            ticker_df = ticker_df.rename(columns={'Adj Close':ticker})
            ticker_df = ticker_df.set_index('Date')
            prices_df = pd.concat([prices_df, ticker_df], axis="columns", sort=True)
            
    return prices_df

## Function: Calc percentage Change, and return TOP 10 best performing TICKERS as DataFrame

In [8]:
def top_10_df(df):
    pct_change = df.pct_change()
    cumulative_returns = (1+pct_change).cumprod() -1
    cum_returns = cumulative_returns.tail(1).T
    column = cum_returns.columns[-1]
    top = cum_returns.sort_values(by=column,ascending=False)
    top = top.drop_duplicates()
    top = top.head(10)
    top = top.reset_index()
    return top

## Function: Return the returns of the TOP DOG Portfolio as DataFrame

In [12]:
def top_10_returns(df,sector,year):
    weights = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
    portfolio_monthly = df.dot(weights)
    my_portfolio_one = (1+ portfolio_monthly).cumprod() -1
    plt.plot(my_portfolio_one)
    plt.xlabel('Time')
    plt.title(f"{sector} 'Top Dog' returns for {year}")
    plt.ylabel('Percent Change')
    plt.savefig(f"{sector}_top_dog_returns{year}.png")
    my_portfolio_one = my_portfolio_one.tail(1)
    my_portfolio_one = pd.DataFrame(portfolio_one)
    my_portfolio_one = my_portfolio_one.reset_index(drop=True)
    my_portfolio_one = my_portfolio_one.rename(columns={0:'Annual Return'})
    my_portfolio_one['Sector'] = sector
    my_portfolio_one['Year'] = year
    my_portfolio_one['Portfolio']= f"{sector} 'Top Dog' for year ending {year}"
    IT_csv = Path(f"./top_{sector}.csv")
    my_portfolio_one.to_csv(path_or_buf=IT_csv, index=False)
    return my_portfolio_one

# Run the code

## grab all ticker data for all sectors

In [13]:
monthly_csv = Path("./aggregatedata.csv")
all_prices_df = pd.read_csv(monthly_csv, parse_dates=True, infer_datetime_format=True)

## Now Loop through sectors

In [ ]:
sectors = ['Technology','Financials','HealthCare']
start = '2015-01-01'
end   = '2017-12-31'
year,month,day = end.split('-')

# Loop through sectors, to find top dogs by sector
for sector in sectors:
    print(f"Calculating Top Portfolio for {sector} for year end {year_end}")
    tickers = get_tickers_for_years(['2015','2016','2017'], sector)
    results_df = combined_df_for_timeframe(all_prices_df,tickers,start,end,sector)
    top10 = top_10_df(results_df)
    top_tickers = []
    top_tickers = top10['index'].tolist()
    top_dog_portfolio = combined_df_for_timeframe(all_prices_df,top_tickers,top_start,top_end,sector)
    print(top_dog_portfolio)
    #print(top_dog_portfolio.plot( title=f"{sector} Portfolio Performance for {year_end}" )   )
    top_10_returns(top_dog_portfolio,sector,year_end)

Calculating Top Portfolio for Technology for year end 2017-12-31


Running combined_df_for_timeframe wtih start: 2015-01-01 and end: 2017-12-31


